# Feature Extraction Demo

This notebook demonstrates feature extraction from GPS signals for spoofing detection.

In [ ]:
import sys
sys.path.append('..')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from src.preprocessing import window_segment, normalize_signal
from src.features import build_feature_vector, compute_cross_correlation, fwhm
from src.utils import plot_correlation

%matplotlib inline

## 1. Generate Synthetic Signal

Create a synthetic GPS-like signal for demonstration.

In [ ]:
# Generate synthetic signal
fs = 5e6
duration = 1.0  # seconds
t = np.arange(0, duration, 1/fs)
signal = np.exp(1j * 2 * np.pi * 1e6 * t) + 0.5 * np.random.randn(len(t)) + 0.5j * np.random.randn(len(t))
print(f"Signal length: {len(signal)} samples")

## 2. Segment Signal

Divide signal into windows for feature extraction.

In [ ]:
window_size = int(0.001 * fs)  # 1 ms windows
overlap = window_size // 2
windows = window_segment(signal, window_size, overlap)
print(f"Number of windows: {len(windows)}")

## 3. Extract Features

Extract features from signal windows.

In [ ]:
# Generate reference code (simple example)
ref_code = np.random.choice([-1, 1], size=window_size)

# Extract features
features_df = build_feature_vector(windows[:10], prn=1, fs=fs, reference_code=ref_code)
print(features_df.head())

## 4. Visualize Correlation

Visualize correlation profile and extracted features.

In [ ]:
# Compute and plot correlation for one window
corr = compute_cross_correlation(windows[0], ref_code)
plot_correlation(corr, fs, title='Correlation Profile Example')

## 5. Feature Statistics

Analyze feature distributions.

In [ ]:
# Display feature statistics
print(features_df.describe())